NOMBRE: Priscilla González


CARNE: 20689


FECHA: 12.10.24


**Objetivo**: La idea de esta práctica es que exploren el desarrollo de un proceso de cuatización. Para esto, deben cubrir lo que se vio en clase:
1. Efectuar el proceso de cuantización.
2. Determinar su error.

In [52]:
import torch

print(torch.__version__)


1.13.1


In [53]:
test_tensor = torch.tensor([[-0.6870,  0.2607, -0.7718,  0.0841],
                       [ 0.6558, -1.1711, -1.0713,  0.2405],
                       [ 1.4967, -0.6928,  0.7961, -1.4614],
                       [ 0.5689, -0.5227, -1.3111, -0.8343]])

In [54]:
rmin = test_tensor.min().item()
rmax = test_tensor.max().item()
qmin = -128
qmax = 127

scale = (rmax - rmin) / (qmax - qmin)
zero_point = round(qmin - (rmin / scale))

In [55]:
print(f"rmin: {rmin}, rmax: {rmax}")
print(f"Scale: {scale}")
print(f"Zero point: {zero_point}")

rmin: -1.461400032043457, rmax: 1.4967000484466553
Scale: 0.011600392472510244
Zero point: -2


In [56]:
quantized_tensor = torch.round(test_tensor / scale + zero_point).clamp(qmin, qmax).to(torch.int8)

In [57]:
deq_tensor = (quantized_tensor.float() - zero_point) * scale

In [58]:
print("Tensor cuantizado:")
print(quantized_tensor)
print("\nTensor descuantizado:")
print(deq_tensor)

Tensor cuantizado:
tensor([[ -61,   20,  -69,    5],
        [  55, -103,  -94,   19],
        [ 127,  -62,   67, -128],
        [  47,  -47, -115,  -74]], dtype=torch.int8)

Tensor descuantizado:
tensor([[-0.6844,  0.2552, -0.7772,  0.0812],
        [ 0.6612, -1.1716, -1.0672,  0.2436],
        [ 1.4965, -0.6960,  0.8004, -1.4616],
        [ 0.5684, -0.5220, -1.3108, -0.8352]])


In [ ]:
#Así como se vio en clase, definan una proceso de linear quantization para test_tensor
#Para llevar esto a acabo, debe encontrar la escala: scale = rmax-rmin/qmax-qmin.
#Trabajando con pytorch int8 (para mapear a int4) qmax=127 y qmin=-128.
#zero_point = qmin-(rmin/scale).
#Con esta informacion ya pueden proceder a trabaja la cuantización lineal.





#Resultado del tensor cuantizado:
quantized_tensor = tensor([[-77, -77, -77, -77],
                           [-77, -77, -77, -77],
                           [-77, -77, -77, -77],
                           [-77, -77, -77, -77]], dtype=torch.int8)
#Resultado del tensor descuantizado:
deq_tensor = tensor([[0., 0., 0., 0.],
                     [0., 0., 0., 0.],
                     [0., 0., 0., 0.],
                     [0., 0., 0., 0.]])


Con la parte anterior correcta, ya pueden proceder a trabajar lo siguiente:

1. Investigar Symmetric mode quantization.
2. per-channel quantization.
3. per-group quantization.

Para cada uno de estos, explicar su funcionamiento y cuando puede ser conveniente usarlos respecto a las otras técnicas.

Finalmente, desarrollar cada uno de la misma manera en que se trabajo la cuantización lineal. (Estos tres son casos especiales de la cuantización lineal, por lo que deberían solo hacer cambios especificos al código de la primera parte).


Responda la siguiente pregunta: ¿Aquí estamos trabajando post-training quantization o quantization-aware training?

### Symmetric Mode Quantization

Para este caso, los valores proceden a cuantizarse, de esta forma se encuentran valores tanto positivos como negativos con respecto a 0. Por lo tanto, los valores mínimos y máximos del tensor original se cuantizan de forma simétrica. Entonces, es bastante conveniente usarlo cuando los valores tienen una distribución centrada alrededor de 0, como se trata ReLU.

 - https://www.ai-bites.net/model-quantization-in-deep-learning/#:~:text=Symmetric%20quantization%20is%20exactly%20what,point%20is%20set%20to%200.

### Per-channel Quantization

Este es comúnmente usado en capas convolucionales ya que cada dimensión del tensor contiene su propia cuantización, en este caso, es el scale y el zero_point.

 - https://medium.com/@curiositydeck/quantization-granularity-aec2dd7a0bb4

### Per-group Quantization

Esta técnica es casi igual a la antrior, solo que en este caso, lo que se hace es que se agrupan varias dimensiones y no solamente 1. Entonces, de igual forma, se le aplica su propia cuantización por dimensión. Esta técnica ayuda bastante cuando se trata de aplicaciones con bastantes capas en redes neuronales.

- https://medium.com/@curiositydeck/quantization-granularity-aec2dd7a0bb4

### Symmetric Mode Quantization

In [59]:
scale_symmetric = max(abs(rmin), abs(rmax)) / (qmax - qmin)
zero_point_symmetric = 0  

quantized_tensor_symmetric = torch.round(test_tensor / scale_symmetric).clamp(qmin, qmax).to(torch.int8)

deq_tensor_symmetric = quantized_tensor_symmetric.float() * scale_symmetric

print("Quantized Tensor:\n", quantized_tensor_symmetric)
print("Dequantized Tensor:\n", deq_tensor_symmetric)


Quantized Tensor:
 tensor([[-117,   44, -128,   14],
        [ 112, -128, -128,   41],
        [ 127, -118,  127, -128],
        [  97,  -89, -128, -128]], dtype=torch.int8)
Dequantized Tensor:
 tensor([[-0.6867,  0.2583, -0.7513,  0.0822],
        [ 0.6574, -0.7513, -0.7513,  0.2406],
        [ 0.7454, -0.6926,  0.7454, -0.7513],
        [ 0.5693, -0.5224, -0.7513, -0.7513]])


### Per-channer Quantization

In [60]:
scales = []
zero_points = []
quantized_tensor_per_channel = torch.empty_like(test_tensor, dtype=torch.int8)

rmin = test_tensor.min().item()
rmax = test_tensor.max().item()
scale = (rmax - rmin) / (qmax - qmin)
zero_point = round(qmin - (rmin / scale))

scales.append(scale)
zero_points.append(zero_point)

quantized_tensor_per_channel = torch.round(test_tensor / scale + zero_point).clamp(qmin, qmax).to(torch.int8)

deq_tensor_per_channel = (quantized_tensor_per_channel.float() - zero_points[0]) * scales[0]

print("Quantized Tensor:\n", quantized_tensor_per_channel)
print("Desquantized Tensor:\n", deq_tensor_per_channel)


Quantized Tensor:
 tensor([[ -61,   20,  -69,    5],
        [  55, -103,  -94,   19],
        [ 127,  -62,   67, -128],
        [  47,  -47, -115,  -74]], dtype=torch.int8)
Desquantized Tensor:
 tensor([[-0.6844,  0.2552, -0.7772,  0.0812],
        [ 0.6612, -1.1716, -1.0672,  0.2436],
        [ 1.4965, -0.6960,  0.8004, -1.4616],
        [ 0.5684, -0.5220, -1.3108, -0.8352]])


### Per-group Quantization

In [61]:
quantized_tensor_per_group = torch.empty_like(test_tensor, dtype=torch.int8) 
deq_tensor_per_group = torch.empty_like(test_tensor, dtype=torch.float32)  

group_size = 2  

for g in range(0, test_tensor.shape[0], group_size):  
    rmin = test_tensor[g:g + group_size, :].min().item()
    rmax = test_tensor[g:g + group_size, :].max().item()
    
    scale = (rmax - rmin) / (qmax - qmin)
    zero_point = round(qmin - (rmin / scale))
    
    quantized_tensor_per_group[g:g + group_size, :] = torch.round(test_tensor[g:g + group_size, :] / scale + zero_point).clamp(qmin, qmax).to(torch.int8)
    
    deq_tensor_per_group[g:g + group_size, :] = (quantized_tensor_per_group[g:g + group_size, :].float() - zero_point) * scale

print("Quantized Tensor:\n", quantized_tensor_per_group)
print("Desquantized Tensor:\n", deq_tensor_per_group)


Quantized Tensor:
 tensor([[ -61,   71,  -73,   47],
        [ 127, -128, -115,   69],
        [ 127,  -62,   67, -128],
        [  47,  -47, -115,  -74]], dtype=torch.int8)
Desquantized Tensor:
 tensor([[-0.6878,  0.2579, -0.7737,  0.0860],
        [ 0.6591, -1.1678, -1.0746,  0.2436],
        [ 1.4965, -0.6960,  0.8004, -1.4616],
        [ 0.5684, -0.5220, -1.3108, -0.8352]])


Se está utilizando post-training potque en este caso el tensor ya está especificado y luego se le aplica la cuantización ya cuando este ya está practicamente entrenado